## Desafio 1: utilizando parâmetros do json_normalize

Ao normalizar o objeto JSON durante a aula passamos simplesmente pd.json_normalize(<dados>) mas o método json_normalize possui diversos parâmetros para trabalhar com estruturas de dados mais complexas.

Você recebeu a tarefa de normalizar o seguinte arquivo JSON chamado “informacoes.json” referente a algumas informações de identificação de um cliente. 

In [170]:
import pandas as pd

dados_informacoes = pd.read_json('C:/Users/Guuh/Downloads/Python para Data Science/Exercicios/8-Pandas-limpeza-tratamento-dados/informacoes.json')
dados_informacoes['enderecos'][0]

{'tipo': 'casa', 'rua': 'Rua A', 'numero': 123, 'cidade': 'São Paulo'}

In [171]:
import json

with open('C:/Users/Guuh/Downloads/Python para Data Science/Exercicios/8-Pandas-limpeza-tratamento-dados/informacoes.json') as f:
    dados_informacoes_brutos = json.load(f)

In [172]:
dados_informacoes_brutos

{'nome': 'JoÃ£o',
 'idade': 28,
 'enderecos': [{'tipo': 'casa',
   'rua': 'Rua A',
   'numero': 123,
   'cidade': 'SÃ£o Paulo'},
  {'tipo': 'trabalho',
   'rua': 'Rua B',
   'numero': 456,
   'cidade': 'Rio de Janeiro'}]}

In [173]:
dados_informacoes_normalizados = pd.json_normalize(dados_informacoes_brutos, record_path='enderecos', meta=['nome', 'idade'], errors='ignore')
dados_informacoes_normalizados

,tipo,rua,numero,cidade,nome,idade
0,casa,Rua A,123,SÃ£o Paulo,JoÃ£o,28
1,trabalho,Rua B,456,Rio de Janeiro,JoÃ£o,28


## Desafio 2: tratando os dados

Sua tarefa é realizar alguns tratamentos e deixar esse arquivo JSON como um Pandas DataFrame, sem dados vazios e com a tipagem das colunas de forma correta. Como isso pode ser feito?

In [174]:
import pandas as pd

dados = pd.read_json('C:/Users/Guuh/Downloads/Python para Data Science/Exercicios/8-Pandas-limpeza-tratamento-dados/dados.json')
dados

,pessoas
0,"{'nome': 'João', 'idade': '25', 'endereco': {'..."
1,"{'nome': 'Maria', 'idade': 30, 'endereco': {'r..."


In [175]:
dados['pessoas'][0]

{'nome': 'João',
 'idade': '25',
 'endereco': {'rua': 'Rua A', 'numero': 123, 'cidade': 'São Paulo'},
 'telefones': ['11 1111-1111', '11 2222-2222']}

In [176]:
import json

with open('C:/Users/Guuh/Downloads/Python para Data Science/Exercicios/8-Pandas-limpeza-tratamento-dados/dados.json') as f:
    dados_brutos = json.load(f)

In [177]:
dados_brutos

{'pessoas': [{'nome': 'JoÃ£o',
   'idade': '25',
   'endereco': {'rua': 'Rua A', 'numero': 123, 'cidade': 'SÃ£o Paulo'},
   'telefones': ['11 1111-1111', '11 2222-2222']},
  {'nome': 'Maria',
   'idade': 30,
   'endereco': {'rua': '', 'numero': 456, 'cidade': 'Rio de Janeiro'},
   'telefones': ['21 3333-3333']}]}

In [178]:
dados_normalizados = pd.json_normalize(dados_brutos, record_path=['pessoas'], errors='ignore')
dados_normalizados

,nome,idade,telefones,endereco.rua,endereco.numero,endereco.cidade
0,JoÃ£o,25,"[11 1111-1111, 11 2222-2222]",Rua A,123,SÃ£o Paulo
1,Maria,30,[21 3333-3333],,456,Rio de Janeiro


In [179]:
dados_normalizados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nome             2 non-null      object
 1   idade            2 non-null      object
 2   telefones        2 non-null      object
 3   endereco.rua     2 non-null      object
 4   endereco.numero  2 non-null      int64 
 5   endereco.cidade  2 non-null      object
dtypes: int64(1), object(5)
memory usage: 224.0+ bytes


In [180]:
dados_normalizados['idade'] = dados_normalizados['idade'].astype(int)

In [181]:
dados_normalizados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nome             2 non-null      object
 1   idade            2 non-null      int32 
 2   telefones        2 non-null      object
 3   endereco.rua     2 non-null      object
 4   endereco.numero  2 non-null      int64 
 5   endereco.cidade  2 non-null      object
dtypes: int32(1), int64(1), object(4)
memory usage: 216.0+ bytes


In [182]:
for col in dados_normalizados.columns:
    print(f"Coluna: {col}")
    if col == 'telefones':
        unique_values = dados_normalizados[col].explode().unique()
    else:
        unique_values = dados_normalizados[col].unique()
    print(unique_values)
    print("-" * 30)


Coluna: nome
['JoÃ£o' 'Maria']
------------------------------
Coluna: idade
[25 30]
------------------------------
Coluna: telefones
['11 1111-1111' '11 2222-2222' '21 3333-3333']
------------------------------
Coluna: endereco.rua
['Rua A' '']
------------------------------
Coluna: endereco.numero
[123 456]
------------------------------
Coluna: endereco.cidade
['SÃ£o Paulo' 'Rio de Janeiro']
------------------------------


In [183]:
dados_sem_vazio = dados_normalizados.loc[dados_normalizados['endereco.rua'] != ''].copy()
dados_sem_vazio

,nome,idade,telefones,endereco.rua,endereco.numero,endereco.cidade
0,JoÃ£o,25,"[11 1111-1111, 11 2222-2222]",Rua A,123,SÃ£o Paulo


In [184]:
dados_sem_vazio['telefone1'] = dados_sem_vazio['telefones'].str[0]
dados_sem_vazio['telefone2'] = dados_sem_vazio['telefones'].str[1]
dados_sem_vazio.drop(columns='telefones', inplace=True)
dados_sem_vazio

,nome,idade,endereco.rua,endereco.numero,endereco.cidade,telefone1,telefone2
0,JoÃ£o,25,Rua A,123,SÃ£o Paulo,11 1111-1111,11 2222-2222


In [185]:
dados_sem_vazio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nome             1 non-null      object
 1   idade            1 non-null      int32 
 2   endereco.rua     1 non-null      object
 3   endereco.numero  1 non-null      int64 
 4   endereco.cidade  1 non-null      object
 5   telefone1        1 non-null      object
 6   telefone2        1 non-null      object
dtypes: int32(1), int64(1), object(5)
memory usage: 60.0+ bytes


## Desafio 3: tratando uma base de dados

Para facilitar a análise dos dados de cadastro de cursos de uma plataforma, você recebeu um arquivo chamado cursos_cadastrados.json. Você foi informado que esses dados podem apresentar problemas que podem prejudicar a análise e interpretação correta dos dados.

Para resolver isso, foi solicitado que você identifique e remova as amostras que apresentam os seguintes problemas:

##### -Valores nulos: são valores ausentes em algumas das colunas que devem ser preenchidos para que a análise seja correta.
##### -Duplicatas: registros iguais que podem prejudicar a análise dos dados, já que estão representando a mesma informação.
##### -Strings vazias: valores de texto que não apresentam nenhum conteúdo escrito, o que pode dificultar a análise de dados, principalmente se a coluna tiver muitos valores assim.
##### -Conversão de tipos: algumas colunas do arquivo podem estar no tipo de dados errado, como texto em vez de número, o que pode prejudicar a análise. Nesse caso você deve realizar a conversão para o tipo correto de cada coluna.
Dentro desse contexto, você precisará de uma abordagem sistemática para limpar os dados. Assim, como isso pode ser feito?

In [186]:
import pandas as pd

dados_cursos = pd.read_json('C:/Users/Guuh/Downloads/Python para Data Science/Exercicios/8-Pandas-limpeza-tratamento-dados/cursos_cadastrados.json')
dados_cursos

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor
0,Introdução à programação,Programação,20 horas,100,2022-01-01,2022-01-20,Curso introdutório à programação com Python,99.9,"{'nome': 'João Silva', 'email': 'joao.silva@em..."
1,Excel para iniciantes,Produtividade,None,None,None,None,None,NaN,"{'nome': 'Maria Oliveira', 'email': 'maria.oli..."
2,Marketing digital para negócios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutório em marketing,89.9,"{'nome': 'Ana Santos', 'email': 'ana.santos@em..."
3,Inteligência artificial,Programação,40 horas,,2022-04-01,,Curso avançado sobre inteligência artificial c...,129.9,"{'nome': '', 'email': 'contato@emailaleatorio...."
4,Inglês para negócios,Idiomas,20 horas,30,,,Curso de inglês para negócios,69.9,"{'nome': 'John Smith', 'email': '', 'telefone'..."
5,Introdução à programação,Programação,20 horas,100,2022-01-01,2022-01-20,Curso introdutório à programação com Python,99.9,"{'nome': 'João Silva', 'email': 'joao.silva@em..."


In [187]:
import json

with open('C:/Users/Guuh/Downloads/Python para Data Science/Exercicios/8-Pandas-limpeza-tratamento-dados/cursos_cadastrados.json') as f:
    dados_cursos_brutos = json.load(f)
dados_cursos_brutos

[{'curso': 'IntroduÃ§Ã£o Ã\xa0 programaÃ§Ã£o',
  'categoria': 'ProgramaÃ§Ã£o',
  'carga_horaria': '20 horas',
  'concluintes': 100,
  'data_inicio': '2022-01-01',
  'data_conclusao': '2022-01-20',
  'descricao': 'Curso introdutÃ³rio Ã\xa0 programaÃ§Ã£o com Python',
  'preco': '99.90',
  'instrutor': {'nome': 'JoÃ£o Silva',
   'email': 'joao.silva@emailaleatorio.com',
   'telefone': '(11) 9999-9999'}},
 {'curso': 'Excel para iniciantes',
  'categoria': 'Produtividade',
  'carga_horaria': None,
  'concluintes': None,
  'data_inicio': None,
  'data_conclusao': None,
  'descricao': None,
  'preco': None,
  'instrutor': {'nome': 'Maria Oliveira',
   'email': 'maria.oliveira@emailaleatorio.com',
   'telefone': '(11) 8888-8888'}},
 {'curso': 'Marketing digital para negÃ³cios',
  'categoria': 'Marketing',
  'carga_horaria': '30 horas',
  'concluintes': 75,
  'data_inicio': '2022-03-01',
  'data_conclusao': '2022-03-31',
  'descricao': 'Curso introdutÃ³rio em marketing',
  'preco': 89.9,
  'ins

In [188]:
dados_cursos_normalizados = pd.json_normalize(dados_cursos_brutos)
dados_cursos_normalizados

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Excel para iniciantes,Produtividade,None,None,None,None,None,None,Maria Oliveira,maria.oliveira@emailaleatorio.com,(11) 8888-8888
2,Marketing digital para negÃ³cios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutÃ³rio em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777
3,InteligÃªncia artificial,ProgramaÃ§Ã£o,40 horas,,2022-04-01,,Curso avanÃ§ado sobre inteligÃªncia artificial...,129.9,,contato@emailaleatorio.com,
4,InglÃªs para negÃ³cios,Idiomas,20 horas,30,,,Curso de inglÃªs para negÃ³cios,69.9,John Smith,,
5,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999


In [189]:
dados_cursos_normalizados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   curso               6 non-null      object
 1   categoria           6 non-null      object
 2   carga_horaria       5 non-null      object
 3   concluintes         5 non-null      object
 4   data_inicio         5 non-null      object
 5   data_conclusao      5 non-null      object
 6   descricao           5 non-null      object
 7   preco               5 non-null      object
 8   instrutor.nome      6 non-null      object
 9   instrutor.email     6 non-null      object
 10  instrutor.telefone  6 non-null      object
dtypes: object(11)
memory usage: 656.0+ bytes


In [190]:
dados_cursos_normalizados.isna().sum()

curso                 0
categoria             0
carga_horaria         1
concluintes           1
data_inicio           1
data_conclusao        1
descricao             1
preco                 1
instrutor.nome        0
instrutor.email       0
instrutor.telefone    0
dtype: int64

In [191]:
dados_cursos_normalizados.isna().sum().sum()

6

In [192]:
dados_cursos_normalizados[dados_cursos_normalizados.isna().any(axis=1)]

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
1,Excel para iniciantes,Produtividade,None,None,None,None,None,None,Maria Oliveira,maria.oliveira@emailaleatorio.com,(11) 8888-8888


In [193]:
dados_cursos_normalizados.loc[dados_cursos_normalizados['curso'] == 'Excel para iniciantes']
#aqui estou verificando se existe mais outro curso de excel para iniciantes para que eu consiga preencher os dados faltantes, aliás o preco do curso seria igual, descrição, etc.

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
1,Excel para iniciantes,Produtividade,None,None,None,None,None,None,Maria Oliveira,maria.oliveira@emailaleatorio.com,(11) 8888-8888


In [194]:
colunas_dropar = ['carga_horaria', 'concluintes', 'data_inicio', 'data_conclusao', 'descricao', 'preco']

In [195]:
dados_cursos_normalizados[colunas_dropar].isna().any(axis=1).sum()

1

In [196]:
df_cursos_sem_nulos = dados_cursos_normalizados.dropna(subset=colunas_dropar).copy()
df_cursos_sem_nulos

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
2,Marketing digital para negÃ³cios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutÃ³rio em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777
3,InteligÃªncia artificial,ProgramaÃ§Ã£o,40 horas,,2022-04-01,,Curso avanÃ§ado sobre inteligÃªncia artificial...,129.9,,contato@emailaleatorio.com,
4,InglÃªs para negÃ³cios,Idiomas,20 horas,30,,,Curso de inglÃªs para negÃ³cios,69.9,John Smith,,
5,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999


In [197]:
df_cursos_sem_nulos.reset_index(drop=True, inplace=True)
df_cursos_sem_nulos

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negÃ³cios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutÃ³rio em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777
2,InteligÃªncia artificial,ProgramaÃ§Ã£o,40 horas,,2022-04-01,,Curso avanÃ§ado sobre inteligÃªncia artificial...,129.9,,contato@emailaleatorio.com,
3,InglÃªs para negÃ³cios,Idiomas,20 horas,30,,,Curso de inglÃªs para negÃ³cios,69.9,John Smith,,
4,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999


In [198]:
df_cursos_sem_nulos.isna().sum()

curso                 0
categoria             0
carga_horaria         0
concluintes           0
data_inicio           0
data_conclusao        0
descricao             0
preco                 0
instrutor.nome        0
instrutor.email       0
instrutor.telefone    0
dtype: int64

In [199]:
df_cursos_sem_nulos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   curso               5 non-null      object
 1   categoria           5 non-null      object
 2   carga_horaria       5 non-null      object
 3   concluintes         5 non-null      object
 4   data_inicio         5 non-null      object
 5   data_conclusao      5 non-null      object
 6   descricao           5 non-null      object
 7   preco               5 non-null      object
 8   instrutor.nome      5 non-null      object
 9   instrutor.email     5 non-null      object
 10  instrutor.telefone  5 non-null      object
dtypes: object(11)
memory usage: 568.0+ bytes


In [200]:
df_cursos_sem_nulos.duplicated()

0    False
1    False
2    False
3    False
4     True
dtype: bool

In [201]:
df_cursos_sem_nulos.duplicated().sum()

1

In [202]:
filtro_duplicadas = df_cursos_sem_nulos.duplicated()
filtro_duplicadas

0    False
1    False
2    False
3    False
4     True
dtype: bool

In [203]:
df_cursos_sem_nulos[filtro_duplicadas]

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
4,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999


In [204]:
df_cursos_sem_nulos.drop_duplicates(inplace=True)

In [205]:
df_cursos_sem_nulos.duplicated().sum()

0

In [206]:
df_cursos_sem_nulos

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negÃ³cios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutÃ³rio em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777
2,InteligÃªncia artificial,ProgramaÃ§Ã£o,40 horas,,2022-04-01,,Curso avanÃ§ado sobre inteligÃªncia artificial...,129.9,,contato@emailaleatorio.com,
3,InglÃªs para negÃ³cios,Idiomas,20 horas,30,,,Curso de inglÃªs para negÃ³cios,69.9,John Smith,,


In [207]:
for col in df_cursos_sem_nulos.columns:
    print(f"Coluna: {col}")
    print(df_cursos_sem_nulos[col].unique())
    print("-" * 30)

Coluna: curso
['IntroduÃ§Ã£o Ã\xa0 programaÃ§Ã£o' 'Marketing digital para negÃ³cios'
 'InteligÃªncia artificial' 'InglÃªs para negÃ³cios']
------------------------------
Coluna: categoria
['ProgramaÃ§Ã£o' 'Marketing' 'Idiomas']
------------------------------
Coluna: carga_horaria
['20 horas' '30 horas' '40 horas']
------------------------------
Coluna: concluintes
[100 75 '' 30]
------------------------------
Coluna: data_inicio
['2022-01-01' '2022-03-01' '2022-04-01' '']
------------------------------
Coluna: data_conclusao
['2022-01-20' '2022-03-31' '']
------------------------------
Coluna: descricao
['Curso introdutÃ³rio Ã\xa0 programaÃ§Ã£o com Python'
 'Curso introdutÃ³rio em marketing'
 'Curso avanÃ§ado sobre inteligÃªncia artificial com Python'
 'Curso de inglÃªs para negÃ³cios']
------------------------------
Coluna: preco
['99.90' 89.9 129.9 69.9]
------------------------------
Coluna: instrutor.nome
['JoÃ£o Silva' 'Ana Santos' '' 'John Smith']
------------------------------
C

In [208]:
#dados_sem_vazio = dados_normalizados[dados_normalizados['Churn'] != ''].copy()
dados_cursos_sem_vazio = df_cursos_sem_nulos[
    (df_cursos_sem_nulos['concluintes'] != '') &
    (df_cursos_sem_nulos['data_inicio'] != '') & 
    (df_cursos_sem_nulos['data_conclusao'] != '') &
    (df_cursos_sem_nulos['instrutor.nome'] != '') &
    (df_cursos_sem_nulos['instrutor.email'] != '') &
    (df_cursos_sem_nulos['instrutor.telefone'] != '')].copy()
dados_cursos_sem_vazio

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negÃ³cios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutÃ³rio em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777


In [209]:
dados_cursos_sem_vazio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   curso               2 non-null      object
 1   categoria           2 non-null      object
 2   carga_horaria       2 non-null      object
 3   concluintes         2 non-null      object
 4   data_inicio         2 non-null      object
 5   data_conclusao      2 non-null      object
 6   descricao           2 non-null      object
 7   preco               2 non-null      object
 8   instrutor.nome      2 non-null      object
 9   instrutor.email     2 non-null      object
 10  instrutor.telefone  2 non-null      object
dtypes: object(11)
memory usage: 192.0+ bytes


In [210]:
dados_cursos_sem_vazio

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20 horas,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.90,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negÃ³cios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutÃ³rio em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777


In [211]:
dados_cursos_sem_vazio['concluintes'] = dados_cursos_sem_vazio['concluintes'].astype(int)

In [212]:
dados_cursos_sem_vazio['data_inicio'] = dados_cursos_sem_vazio['data_inicio'].astype('datetime64[ns]')

In [213]:
dados_cursos_sem_vazio['data_conclusao'] = dados_cursos_sem_vazio['data_conclusao'].astype('datetime64[ns]')

In [214]:
dados_cursos_sem_vazio['preco'] = dados_cursos_sem_vazio['preco'].astype(float)

In [215]:
dados_cursos_sem_vazio['carga_horaria'] = dados_cursos_sem_vazio['carga_horaria'].str.replace('[a-zA-z]', '', regex=True)
dados_cursos_sem_vazio

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.9,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negÃ³cios,Marketing,30,75,2022-03-01,2022-03-31,Curso introdutÃ³rio em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777


In [216]:
dados_cursos_sem_vazio['carga_horaria'] = dados_cursos_sem_vazio['carga_horaria'].astype(int)

In [217]:
dados_cursos_sem_vazio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   curso               2 non-null      object        
 1   categoria           2 non-null      object        
 2   carga_horaria       2 non-null      int32         
 3   concluintes         2 non-null      int32         
 4   data_inicio         2 non-null      datetime64[ns]
 5   data_conclusao      2 non-null      datetime64[ns]
 6   descricao           2 non-null      object        
 7   preco               2 non-null      float64       
 8   instrutor.nome      2 non-null      object        
 9   instrutor.email     2 non-null      object        
 10  instrutor.telefone  2 non-null      object        
dtypes: datetime64[ns](2), float64(1), int32(2), object(6)
memory usage: 176.0+ bytes


In [218]:
dados_cursos_sem_vazio

,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,IntroduÃ§Ã£o Ã programaÃ§Ã£o,ProgramaÃ§Ã£o,20,100,2022-01-01,2022-01-20,Curso introdutÃ³rio Ã programaÃ§Ã£o com Python,99.9,JoÃ£o Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negÃ³cios,Marketing,30,75,2022-03-01,2022-03-31,Curso introdutÃ³rio em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777


## Desafio 4: identificando fraudes


Foi informado que nessa base de dados há possíveis fraudes que podem ser identificadas através dos outliers no “valor da transação”. Basicamente a fraude seria um outlier.

Sua missão é construir uma função que recebe dois parâmetros, a coluna que você quer analisar se tem outlier e o DataFrame em questão. A intenção de fazer isso é aproveitar novamente essa função tanto para outras colunas quanto para outros DataFrames futuramente em outras análises. O método que deve ser implementado para identificação do outlier é do intervalo interquartil.

Como podemos fazer isso?

In [238]:
def indentificar_outliers(coluna, dataframe):
    Q1 = dataframe[coluna].quantile(.25)
    Q3 = dataframe[coluna].quantile(.75)
    IQR = Q3-Q1
    limite_inferior = Q1 - 1.5*IQR
    limite_superior = Q3 + 1.5*IQR

    #acha os indices que contenham possíveis outliers
    outliers = (dataframe[coluna] < limite_inferior) | (dataframe[coluna] > limite_superior)
    outliers = dataframe[outliers][coluna]
    return outliers

In [239]:
import pandas as pd

# criando um DataFrame com 30 transações aleatórias
df = pd.DataFrame({
    'ID da transação': range(1, 31),
    'Valor da transação': [100, 200, 150, 500, 300, 913, 250, 400, 200, 150, 
                           200, 200, 400, 300, 150, 301, 805, 300, 400, 250, 
                           150, 100, 500, 600, 200, 350, 100, 250, 800, 250],
    'Data da transação': pd.date_range(start='2022-01-01', end='2022-01-30', freq='D'),
    'Local da transação': ['São Paulo, Brasil', 'Rio de Janeiro, Brasil', 'Belo Horizonte, Brasil', 'São Paulo, Brasil', 
                           'São Paulo, Brasil', 'Nova Iorque, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'Rio de Janeiro, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Los Angeles, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Miami, EUA', 'São Paulo, Brasil']
})

In [240]:
indentificar_outliers('Valor da transação', df)

5     913
16    805
28    800
Name: Valor da transação, dtype: int64